This notebook calculates the sediment contributing drainage area in the year 2025 for all of ResNet using the methods of Minear and Kondolf (xxxx).

In [89]:
#import packages
import pandas as pd
import numpy as np
import os
from datetime import datetime
import ast


pd.set_option('display.max_columns',None)

In [91]:
#load data

# ResNet File location and name
today = datetime.today().strftime('%Y%m%d')
resnet_orig = pd.read_csv(f'Outputs/ResNet_{today}.csv') #if running same date
# resnet_orig = pd.read_csv(f'Outputs/ResNet_20250601.csv') #if want to input manual date

#load canadian dams for calculating
canada = pd.read_csv('Inputs/InputCanada.csv')

#output file location
out_folder = 'Outputs' 

In [93]:
#conversions
convert1 = 1233.482 #converts m3 to ac-ft is convert1*AF=m3, or from m3 is m3/convert1=AF
convert2 = 2.59 #converts km2 and mi2 is convert2*mi2=km2 or from km2 is km2/convert2=mi2

In [95]:
#combine Canada and resnet
resnet = pd.concat([resnet_orig,canada],ignore_index=True)

resnet = resnet.sort_values(by='ShortID',ascending = True)

In [97]:
resnet['yrr'] = resnet['yrr'].replace(0,np.nan) #replace yrr with nan for canadian dams

In [99]:
#Manually add major Canadian dams to the routing in the Columbia River basin for calculating sediment contributing drainage area

## Boundary Dam
resnet.loc[resnet.ShortID==117361, 'ToDam'] = 500005 #route to a Canadian dam, not directly to Grand Coulee
resnet.loc[resnet.ShortID==117361, 'GRanDTag'] = 500005

## Libby Dam
resnet.loc[resnet.ShortID==78196, 'ToDam'] = 500003 #route to a Canadian dam, not directly to Grand Coulee
resnet.loc[resnet.ShortID==78196, 'GRanDTag'] = 500003

## Grand Coulee Dam
resnet.loc[resnet.ShortID==117548, 'FromDam'] = resnet.loc[resnet.ShortID==117548, 'FromDam'].apply(
lambda x: [num for num in x if num not in {117361,78196}]) #remove Libby and Boundary ShortIDs from Coulee FromDam

resnet.loc[resnet.ShortID==117548, 'FromDam'] = resnet.loc[resnet.ShortID==117548, 'FromDam'].apply(
lambda x: x + [500002, 500004, 500006]) #Add Canadian dams to Coulee FromDam


In [101]:
#Remove rivers from ToDam so only dams are routed to dams.

damstoriver = resnet.loc[resnet.ToDam<0]

while len(damstoriver)>0:
    for i in range(len(damstoriver)):
        #find a dam that goes to a river instead of another dam
        damlocationtofix = damstoriver.index[i] #index of the dam to fix
        river = resnet.loc[damlocationtofix,'ToDam'] #ShortID of the river it goes to
        riverloc = resnet.loc[resnet.ShortID==river].index #location of the river
        rivertodam = resnet.loc[riverloc,'ToDam'] #the ToDam of the river
        resnet.loc[damlocationtofix,'ToDam'] = rivertodam.iloc[0] #replace ToDam of target dam with ToDam of river
    damstoriver = resnet.loc[resnet.ToDam<0]


In [102]:
#  now that to dam has been replaced on real dams, remove to dam from rivers
resnet.loc[resnet.IsRiverMth==1,'ToDam'] = np.nan

#fix terminal dam flag after removing rivers
resnet.loc[resnet.ToDam.isna(),'flagTerm'] = 1
resnet.loc[resnet.IsRiverMth==1, 'flagTerm'] = np.nan


In [103]:
#Ranking dams and assigning dam order (similar to stream order). This does not include Rivers.

Rank = np.full(len(resnet['ShortID']),np.nan)
Rank[np.where(resnet.flagHW==1)[0]] = 1
Rank[np.where(resnet.IsRiverMth == 1)[0]] = 0

Dam1 = []
Dam2 = []
DA1 = []
DA2 = []

DAerrorNumber = 0;
i = 1
ranknum= np.where(Rank == i)[0]
while len(ranknum)>0:
    for j in range(len(ranknum)):
        thisdam = ranknum[j]

        #process nonterminal dams
        if resnet.iloc[thisdam].flagTerm == 0: #if the dam is not a terminal dam
            thisdamDA = resnet.iloc[thisdam].DivDASqKM #identify the drainage area of dam j
            thatdam = np.where(resnet.ShortID == resnet.iloc[thisdam].ToDam)[0][0] #find the downstream dam
            thatdamDA = resnet.iloc[thatdam].DivDASqKM #get the drainage area of the downstream dam

            #make sure the downstream DA doesn't exceed the upstream DA
            if (thatdamDA<thisdamDA):
                DAerrorNumber = DAerrorNumber+1
                
            else:
                Rank[thatdam] = Rank[thisdam]+1
    i = i+1
    ranknum = np.where(Rank == i)[0]

if DAerrorNumber != 0:
    print("Drainage area error: manually investigate drainage areas in GIS")

resnet['Rank']=Rank

DAerrors = pd.DataFrame({'Dam1': Dam1, 'Dam2': Dam2, 'DA1': DA1, 'DA2': DA2})


In [104]:
#Get rid of rivers
resnet = resnet.loc[resnet['IsRiverMth'] != 1]


In [105]:
#For unreasonable or nonexistent yrc, replace with the 90th percentile of when dams were built; earlier than 1700 or later than 2024

# Identify valid values (1700 ≤ yrc ≤ 2024)
valid_yrc = resnet.loc[(resnet['yrc'] >= 1700) & (resnet['yrc'] <= 2024), 'yrc']

# Compute the 90th percentile from valid values
percentile_90 = np.percentile(valid_yrc, 90)

# Replace out-of-range values with the 90th percentile
resnet.loc[(resnet['yrc'] < 1700) | (resnet['yrc'] > 2024), "yrc"] = percentile_90


In [108]:
#create the timeseries
t = np.arange(1699, 2051) #run from 1699 to 2050
numdam=len(resnet.ShortID); #number of dams
numt=len(t); #length of time

#create empty variables
capcalc = np.full((len(resnet.ShortID), numt), np.nan) #this is a variable that would need to be stored in the structure
sedshed = np.full((len(resnet.ShortID), numt), np.nan) #this is the km2 area of the watershed that has sediment getting trapped in reservoir (so contribut DA * trap efficiency)
calctrap = np.full((len(resnet.ShortID),numt),0.0)
wsedshed = np.full((len(resnet.ShortID), 1), np.nan) #effective sediment contributing DA
origDA = np.full((len(resnet.ShortID), numt), np.nan) #original drainage area through time

for j in range(len(resnet.ShortID)):
    origDA[j,:] = resnet.iloc[j].DivDASqKM

wSAatdam = np.full((len(resnet.ShortID), 1), np.nan) #Time-weighted sediment-contributing drainage area above reservoir X
AveTrap = np.full((len(resnet.ShortID), 1), np.nan) #time-weighted trap efficiency
wseddel = np.full((len(resnet.ShortID), 1), np.nan) #m3, total volume of sediment delivered to reservoir X between time 1 and time 2
wSDR = np.full((len(resnet.ShortID), 1), np.nan) #m3/yr, sediment delivery rate, mean volume of sediment delivered to reservoir X per year between time 1 and time 2
wSDRyield = np.full((len(resnet.ShortID), 1), np.nan) #m3/(km3*t), sediment yield, volume of sediment per km2 per year

sedDAtoDS = np.full((len(resnet.ShortID), numt), np.nan) #drainage area that moves downstream
SAatdam = origDA #sediment contributing drainage area upstream from reservoir X (does not include trap efficiency at reservoir X)

In [109]:
# Trap efficiency

#for kappa: coarse (sand) = 1, medium (silt) = 0.1, fine (clay) = 0.046
#we use the design assumption for reservoirs of silt
kappa = np.full((len(resnet.ShortID), 1), 0.1)

#setting initial trap efficiency. Before and after the dam is in place it is zero. While the dam exists, calculate a value.
#This trap efficiency is static through time based on the initial TE.
for j in range(len(resnet.ShortID)):
    #assign trap efficiency as 0 before dam completion
    yrc = resnet.iloc[j].yrc
    yrr = resnet.iloc[j].yrr
    
    
    # Assuming 't' is a NumPy array and 'data' is a pandas DataFrame
    predam = np.where(t < yrc)[0]

    # Handling cases where yrr might be NaN
    if np.isnan(yrr):
        postdam = np.where(t >= yrc)[0]
        removed = np.array([])  # No dams removed
    else:
        postdam = np.where((t >= yrc) & (t < yrr))[0]
        removed = np.where(t >= yrr)[0]

        calctrap[j, removed] = 0
        capcalc[j, removed] = 0

    # Initialize arrays (assuming calctrap and capcalc exist)
    calctrap[j, predam] = 0
    capcalc[j, predam] = 0

    origDA[j, :] = resnet.iloc[j].DivDASqKM

    calctrap[j, postdam] = 1 - 1. / (1 + kappa[j] * ((resnet.iloc[j].MaxStor_m3 / convert1) / (resnet.iloc[j].DivDASqKM / convert2)))

    capcalc[j, postdam] = resnet.iloc[j].MaxStor_m3

    # Since static, take the first trap efficiency for AveTrap
    AveTrap[j, 0] = calctrap[j, postdam[0]]      
 

for i in range(int(max(resnet.Rank))):
    ranknum = np.where(resnet.Rank == i)[0]
    jmax = len(ranknum)
    sedshed[ranknum,:] = SAatdam[ranknum,:] * calctrap[ranknum,:] #km2, this will calculate sedshed for the rank we are on
    sedDAtoDS[ranknum,:] = SAatdam[ranknum,:] - sedshed[ranknum,:] #km2, this is the volume moving downstream past the dam in a given year

    for j in range(jmax):
        val = ranknum[j]
        if resnet.iloc[val].flagTerm == 0: #If it isn't a terminal dam, move the drainage area downstream
            todam = np.where(resnet.ShortID == resnet.iloc[val].ToDam)
            SAatdam[todam,:] = SAatdam[todam,:] - (resnet.iloc[val].DivDASqKM - sedDAtoDS[val,:]) #Adjusts SA at dam for next time loop. Rank of this dam must be higher than dam it comes from
                
            #double check for any drainage area errors that can be caused by flow diversions
            DAerror = np.where(SAatdam[todam,:]<0)[0]
            if len(DAerror)>0:
                print('Drainage area error! Upstream drainage area is larger than downstream drainage area.')


In [110]:
#Convert output to a dataframe
df = pd.DataFrame(SAatdam, columns=t)

In [111]:
## Rejoin to the original dataset to add rivers back in and remove Canadian dams used in SCA routing.

#SA at dam
resnet['SCA2025'] = df[2025]
# join back to original so keeps original ToDam. Adds field for Rank and SCA 2025. Transfer over flagTerm

resnet_orig = resnet_orig.drop('flagTerm',axis=1)

final = pd.merge(resnet_orig,resnet[['ShortID','flagTerm','Rank','SCA2025']],on='ShortID',how='left')

#save file
final.to_csv(f'Outputs/ResNet_SCA_{today}.csv')